# Double Header Issue

After collecting all gamelog TXT files from Retrosheet Game Logs, in order to merge this data with the main data set I needed to create a feature to account for double headers. The keys for the merging are date and home team, but on days with double headers, these keys are not unique. Thus, this notebook creates features to account for doubleheaders (and in one case a triple header) to ensure that each observation has a unique key to merge on. 

In [2]:
import pandas as pd
import json
import glob
import numpy as np

In [2]:
#read in dataframe created from Retrosheet Gamelogs
game_logs = pd.read_csv('data/retrograde_gamelog.csv')

In [3]:
#drop column with no data
game_logs = game_logs.drop(columns = ['Unnamed: 6'])

In [4]:
game_logs.head()

,date,away_team,home_team,game_time,park_id,attendance
0,20080325,BOS,OAK,N,TOK01,44628.0
1,20080326,BOS,OAK,N,TOK01,44735.0
2,20080330,ATL,WAS,N,WAS11,39389.0
3,20080331,PIT,ATL,N,ATL02,45269.0
4,20080331,MIL,CHN,D,CHI11,41089.0


In [5]:
#change format of date variable from string to datetime object
game_logs['date'] = pd.to_datetime(game_logs['date'], format = '%Y%m%d')

In [6]:
#sort by date
game_logs = game_logs.sort_values(by = ['date'])

In [7]:
#reset index
game_logs = game_logs.reset_index().drop(columns = ['index'])

In [8]:
game_logs.head()

,date,away_team,home_team,game_time,park_id,attendance
0,1900-04-19,PHI,BSN,NaN,BOS05,10000.0
1,1900-04-19,CHN,CIN,NaN,CIN05,12000.0
2,1900-04-19,BRO,NY1,NaN,NYC10,15000.0
3,1900-04-19,PIT,SLN,NaN,STL05,12000.0
4,1900-04-20,BRO,NY1,NaN,NYC10,NaN


To determine which game observations were played on double header days, I grouped all observations by date, determined which dates have multiple observations of the same home team, determined the indices of these observations, then created binary variables is_double_header and is_triple_header to denote which games were the second game of a double header or the third game of a triple header (there is only one).

In [9]:
#group by date and count home teams per date
double_headers = game_logs.groupby('date').home_team.value_counts()
double_headers = double_headers[double_headers > 1]
triple_headers = double_headers[double_headers > 2]

In [12]:
triple_headers

date        home_team
1920-10-02  PIT          3
Name: home_team, dtype: int64

In [13]:
double_headers

date        home_team
1900-05-30  BRO          2
            BSN          2
            NY1          2
            PHI          2
1900-07-04  CHN          2
                        ..
2018-09-11  PHI          2
2018-09-13  NYN          2
2018-09-26  BOS          2
2018-09-28  MIN          2
2018-09-29  BAL          2
Name: home_team, Length: 15606, dtype: int64

In [14]:
#create dummy variables is_double_header and is_triple_header
game_logs = game_logs.assign(is_double_header = 0)
game_logs = game_logs.assign(is_triple_header = 0)

In [22]:
game_logs.head()

,date,away_team,home_team,game_time,park_id,attendance,is_double_header,is_triple_header
0,1900-04-19,PHI,BSN,NaN,BOS05,10000.0,0,0
1,1900-04-19,CHN,CIN,NaN,CIN05,12000.0,0,0
2,1900-04-19,BRO,NY1,NaN,NYC10,15000.0,0,0
3,1900-04-19,PIT,SLN,NaN,STL05,12000.0,0,0
4,1900-04-20,BRO,NY1,NaN,NYC10,NaN,0,0


In [15]:
#create array with date and home team name for each double header game played
all_double_headers_home = []
for j in range(len(double_headers)):
    home_team = [double_headers.index[j][0], double_headers.index[j][1]]
    all_double_headers_home.append(home_team)

In [22]:
#function to change value of new features for second (or third) games on one day. Takes the gamelog dataframe and 
#the list of dates and home team names as an argument, and changes the value of the respective dummy variables
#at each index of a double header or triple header
def assign_multigame_values(df, team_list):
    for entry in team_list:
        games = df[(df.date == entry[0]) & (df.home_team == entry[1])]
        indices = []
        for j in range(len(games)):
            indices.append(games.index[j])
        if len(indices) == 2:
            df.at[indices[1], 'is_double_header'] = 1
        elif len(indices) == 3:
            df.at[indices[1], 'is_double_header'] = 1
            df.at[indices[2], 'is_triple_header'] = 1
        else:
            print(games)

In [24]:
assign_multigame_values(game_logs, all_double_headers_home)

In [27]:
game_logs[game_logs.is_triple_header == 1]

,date,away_team,home_team,game_time,park_id,attendance,is_double_header,is_triple_header
25882,1920-10-02,CIN,PIT,D,PIT06,NaN,0,1


In [28]:
#write to csv
game_logs.to_csv('data/retrograde_gamelog.csv')

In [5]:
#read in mlb_elo dataframe
mlb_elo = pd.read_csv('data/mlb_final.csv', \
                     low_memory = False, index_col = [0])

In [6]:
mlb_elo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 198286 entries, 0 to 198285
Data columns (total 57 columns):
date                    198286 non-null object
season                  198286 non-null int64
neutral                 198286 non-null int64
playoff                 1608 non-null object
team1                   198286 non-null object
team2                   198286 non-null object
elo1_pre                198286 non-null float64
elo2_pre                198286 non-null float64
elo_prob1               198286 non-null float64
elo_prob2               198286 non-null float64
elo1_post               198286 non-null float64
elo2_post               198286 non-null float64
rating1_pre             198286 non-null float64
rating2_pre             198286 non-null float64
pitcher1                198278 non-null object
pitcher2                198278 non-null object
pitcher1_rgs            184642 non-null float64
pitcher2_rgs            184642 non-null float64
pitcher1_adj            183965 non-nu

In [7]:
#assign binary features for double header and triple header
mlb_elo = mlb_elo.assign(is_double_header = 0)
mlb_elo = mlb_elo.assign(is_triple_header = 0)

In [8]:
mlb_elo.head()

,date,season,neutral,playoff,team1,team2,elo1_pre,elo2_pre,elo_prob1,elo_prob2,...,WSF1,AWND,PSUN,WSF2,WSF5,latitude,longitude,ACSC,is_double_header,is_triple_header
0,1900-04-19,1900,0,NaN,STL,PIT,1503.240,1516.607,0.515297,0.484703,...,NaN,NaN,NaN,NaN,NaN,38.662778,90.222222,NaN,0,0
1,1900-04-19,1900,0,NaN,ATL,PHI,1550.730,1536.910,0.554213,0.445787,...,NaN,NaN,NaN,NaN,NaN,42.337500,71.086944,NaN,0,0
2,1900-04-19,1900,0,NaN,SFG,LAD,1476.101,1541.571,0.440602,0.559398,...,NaN,NaN,NaN,NaN,NaN,40.798056,73.950278,NaN,0,0
3,1900-04-19,1900,0,NaN,CIN,CHC,1519.974,1507.474,0.552335,0.447665,...,NaN,NaN,NaN,NaN,NaN,41.511389,81.644167,NaN,0,0
4,1900-04-20,1900,0,NaN,SFG,LAD,1474.940,1542.732,0.437310,0.562690,...,NaN,NaN,NaN,NaN,NaN,40.798056,73.950278,NaN,0,0


With the mlb_elo dataset read in, I will repeat the same process as above to account for double and triple headers.

In [10]:
#group by date and count home teams
double_headers_elo = mlb_elo.groupby('date').team1.value_counts()
double_headers_elo = double_headers_elo[double_headers_elo > 1]
triple_headers_elo = double_headers_elo[double_headers_elo > 2]

In [13]:
#generate list of dates and home teams
all_double_headers_home_elo = []
for j in range(len(double_headers_elo)):
    home_team = [double_headers_elo.index[j][0], double_headers_elo.index[j][1]]
    all_double_headers_home_elo.append(home_team)

In [19]:
#function to alter values in dataframe
assign_multigame_values(mlb_elo, all_double_headers_home_elo)

In [39]:
pd.set_option('max.columns', 100)
mlb_elo.head()

,date,season,neutral,playoff,team1,team2,elo1_pre,elo2_pre,elo_prob1,elo_prob2,elo1_post,elo2_post,rating1_pre,rating2_pre,pitcher1,pitcher2,pitcher1_rgs,pitcher2_rgs,pitcher1_adj,pitcher2_adj,rating_prob1,rating_prob2,rating1_post,rating2_post,score1,score2,team_code,team,state_code,year,primary_stadium,primary_latitude,primary_longitude,secondary_stadium,secondary_latitude,secondary_longitude,attendance/game,pitching_park_factor,batting_park_factor,station_id,TMAX,TMIN,PRCP,SNOW,SNWD,ACSH,WSFG,WSFI,WSFM,WSF1,AWND,PSUN,WSF2,WSF5,latitude,longitude,ACSC,is_double_header,is_triple_header,team_name,rg_code
0,1900-04-19,1900,0,NaN,STL,PIT,1503.240,1516.607,0.515297,0.484703,1505.258,1514.589,1503.335,1516.682,leevs101,younc102,NaN,NaN,NaN,NaN,0.516129,0.483871,1505.333,1514.684,3,0,STL,St. Louis Cardinals,MO,1900.0,Robison Field,38.662778,90.222222,NaN,NaN,NaN,"3,750",98.0,98.0,USW00093963,211.0,94.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.662778,90.222222,NaN,0,0,St. Louis Cardinals,SLN
1,1900-04-19,1900,0,NaN,ATL,PHI,1550.730,1536.910,0.554213,0.445787,1548.944,1538.697,1550.654,1536.878,ortha101,willv101,NaN,NaN,NaN,NaN,0.554151,0.445849,1548.856,1538.675,17,19,ATL,Boston Beaneaters,MA,1900.0,South End Grounds III,42.337500,71.086944,NaN,NaN,NaN,"2,767",111.0,111.0,USW00094701,233.0,111.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.337500,71.086944,NaN,0,0,Boston Beaneaters,BSN
2,1900-04-19,1900,0,NaN,SFG,LAD,1476.101,1541.571,0.440602,0.559398,1474.940,1542.732,1476.107,1541.625,kennb101,carrb102,NaN,NaN,NaN,NaN,0.443621,0.556379,1474.944,1542.788,2,3,SFG,New York Giants,NY,1900.0,Polo Grounds IV,40.798056,73.950278,NaN,NaN,NaN,"2,676",97.0,95.0,USC00305540,122.0,56.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.798056,73.950278,NaN,0,0,New York Giants,NY1
3,1900-04-19,1900,0,NaN,CIN,CHC,1519.974,1507.474,0.552335,0.447665,1517.588,1509.859,1519.960,1507.347,grifc101,philb101,NaN,NaN,NaN,NaN,0.555300,0.444700,1517.559,1509.748,10,13,CIN,Cincinnati Reds,OH,1900.0,League Park II,41.511389,81.644167,NaN,NaN,NaN,"2,698",99.0,97.0,USC00331662,178.0,89.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.511389,81.644167,NaN,0,0,Cincinnati Reds,CIN
4,1900-04-20,1900,0,NaN,SFG,LAD,1474.940,1542.732,0.437310,0.562690,1477.816,1539.856,1474.944,1542.788,kitsf101,seymc101,NaN,NaN,NaN,NaN,0.437236,0.562764,1477.820,1539.912,12,8,SFG,New York Giants,NY,1900.0,Polo Grounds IV,40.798056,73.950278,NaN,NaN,NaN,"2,676",97.0,95.0,USC00305540,139.0,83.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.798056,73.950278,NaN,0,0,New York Giants,NY1


Unfortunately, the Retrosheet and mlb_elo dataframes do not use the same team codes and and thus cannot be merged on that key. To work around this, in the retrosheet_gamelogs.ipynb notebook I created a csv mapping mlb_elo team names to the respective Retrosheet team codes. I will load that .csv in as a dataframe and merge it with the mlb_elo dataframe and then use this team code as a key in the final merge. 

In [26]:
rg_codes = pd.read_csv('data/rg_codes_by_team.csv', index_col = [0])

In [29]:
rg_codes.head()

,team_name,rg_code
0,Tampa Bay Rays,TBA
1,Tampa Bay Devil Rays,TBA
2,Oakland Athletics,OAK
3,Kansas City Athletics,KC1
4,Philadelphia Athletics,PHA


In [28]:
#check number of team codes is the same in each frame
print(len(rg_codes.team_name.unique()), len(mlb_elo.team.unique()))

59 59


In [30]:
#merge Retrosheet team codes to mlb_elo frame 
mlb_elo = mlb_elo.merge(rg_codes, how = 'left', left_on = 'team', right_on = 'team_name')

In [32]:
mlb_elo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 198286 entries, 0 to 198285
Data columns (total 61 columns):
date                    198286 non-null object
season                  198286 non-null int64
neutral                 198286 non-null int64
playoff                 1608 non-null object
team1                   198286 non-null object
team2                   198286 non-null object
elo1_pre                198286 non-null float64
elo2_pre                198286 non-null float64
elo_prob1               198286 non-null float64
elo_prob2               198286 non-null float64
elo1_post               198286 non-null float64
elo2_post               198286 non-null float64
rating1_pre             198286 non-null float64
rating2_pre             198286 non-null float64
pitcher1                198278 non-null object
pitcher2                198278 non-null object
pitcher1_rgs            184642 non-null float64
pitcher2_rgs            184642 non-null float64
pitcher1_adj            183965 non-nu

In [34]:
#convert date to datetime object for merge
mlb_elo['date'] = pd.to_datetime(mlb_elo['date'], format = '%Y-%m-%d')

In [43]:
#merge dataframes on date, home team, is_double_header and is_triple_header
mlb_elo = mlb_elo.merge(game_logs, how = 'left', left_on = ['date', 'rg_code', 'is_double_header',
                                                                'is_triple_header'],
                       right_on = ['date', 'home_team', 'is_double_header', 'is_triple_header'])

In [44]:
mlb_elo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 198286 entries, 0 to 198285
Data columns (total 66 columns):
date                    198286 non-null datetime64[ns]
season                  198286 non-null int64
neutral                 198286 non-null int64
playoff                 1608 non-null object
team1                   198286 non-null object
team2                   198286 non-null object
elo1_pre                198286 non-null float64
elo2_pre                198286 non-null float64
elo_prob1               198286 non-null float64
elo_prob2               198286 non-null float64
elo1_post               198286 non-null float64
elo2_post               198286 non-null float64
rating1_pre             198286 non-null float64
rating2_pre             198286 non-null float64
pitcher1                198278 non-null object
pitcher2                198278 non-null object
pitcher1_rgs            184642 non-null float64
pitcher2_rgs            184642 non-null float64
pitcher1_adj            18396

In [45]:
mlb_elo.to_csv('data/mlb_final_retro.csv')